In [1]:
print("hello")

hello


Approaches:

1. Chains in whihc we execute retreieval step
2. Agents, where we give LLM discretion over whether and how to execute a retreival step (or multiple steps)

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq

In [4]:
groq_api_key = os.getenv("GROQ_API_Key")

In [5]:
llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D6E63F3E20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D6E642D2D0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
# os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
#from langchain_huggingface import HuggingFaceEmbedings
# embeddings =  HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [17]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [15]:
embeddings = OllamaEmbeddings(model='gemma:2b')

In [10]:
import bs4
loader = WebBaseLoader(
    web_paths=("https://python.langchain.com/docs/tutorials/agents/",),

)

docs=loader.load()

In [11]:
docs

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/agents/', 'title': 'Build an Agent | 🦜️🔗 LangChain', 'description': "By themselves, language models can't take actions - they just output text.", 'language': 'en'}, page_content='\n\n\n\n\nBuild an Agent | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [13]:
texts

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/agents/', 'title': 'Build an Agent | 🦜️🔗 LangChain', 'description': "By themselves, language models can't take actions - they just output text.", 'language': 'en'}, page_content='Build an Agent | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/agents/', 'title': 'Build an Agent | 🦜️🔗 LangChain', 'description': "By themselves, language models can't take actions - they just output text.", 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Genera

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002D6FF762C80>, search_kwargs={})

In [28]:
system_prompt = (
    "You are a helpful assistant that answers questions about LangChain. "
    "You have access to the following documents:"
    "\n{context}\n"
    "Use the information in the documents to answer the user's question. "
    "\n\n"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [29]:
from langchain.chains import create_retrieval_chain


In [30]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What are Langchain components?"})

In [35]:
response['answer']

'According to the provided documentation, LangChain components include:\n\n1. Language Models (LLMs): LangChain supports multiple language models that can be used interchangeably.\n2. Agents: Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action.\n3. Tool-Calling: Agents can call tools to execute actions and receive results, which can then be fed back into the LLM to determine whether more actions are needed or whether it is okay to finish.\n\nAdditionally, the documentation mentions the following components:\n\n4. Configurable components: These allow users to customize the behavior of their LangChain applications.\n5. LangSmith: This is a utility for inspecting the inner workings of LangChain applications, which can be useful for debugging complex applications.\n\nIt\'s worth noting that the provided code snippet showcases an "end-to-end agent" that uses an LLM to decide which tools to use, and it has co

# Chat History

In [37]:
from langchain.chains import create_history_aware_retriever# retriever knowledge of previous questions history
from langchain_core.prompts import MessagesPlaceholder # to use in the prompt for a variable that will hold the history of previous questions

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might be of reference context in chat history,"
    "contextualize the question to be more specific."
    "without chat history, do not answer the question"
    "just reformalize it if needed and otherwise return the question as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"), # Placeholder for chat history
    ]
)




In [41]:
history_aware_retriever = create_history_aware_retriever(retriever,llm,contextualize_q_prompt)

In [42]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D6E63F3E20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D6E642D2D0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********')))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.

In [46]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"), # Placeholder for chat history
    ]
)

In [47]:
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
"""from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

chat_history = []

question = "What are Langchain components?"
response_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response_1['answer'])
    ]
)

question2 = "What are the different types of agents in Langchain?"
response_2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})

print(response_2['answer'])"""

from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="What is Self-Reflection"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1['answer'])
    ]
)

question2="Tell me more about it?"
response2=rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])